## Fuzzy logic model related with socioeconomics and tourism
##### Authors: Alejandra Palacio, José Manuel Ramírez, Juan José Wilches y Carlos Gustavo Vélez.

In [1]:
# To import libraries
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
file = 'Data Fuzzy Models.xlsx'

df = pd.read_excel(file, sheet_name= 'Socioeconomical')

data_dicts = df.to_dict(orient='records')

commune_data = {row['Código']: {
                            'Nombre': row['Nombre'],
                            'Identificador': row['Identificador'],
                            'Indicador turismo': row['Indicador turismo'],
                            'Violencia de género': row['Violencia de género'],
                            'IDH': row['IDH'],
                            'Tasa de desempleo': row['Tasa de desempleo'],
                            'Turismo sostenible': row.get('Turismo sostenible', None)}  
               for index, row in df.iterrows()}

In [3]:
# Especifica el archivo Excel y la hoja que quieres leer
file = 'Data Fuzzy Models.xlsx'

# Lee los datos de la hoja en un DataFrame
df = pd.read_excel(file, sheet_name= 'Socioeconomical')

# Convertir el DataFrame a una lista de diccionarios
data_dicts = df.to_dict(orient='records')

print(data_dicts)

[{'Código': 1, 'Nombre': 'Popular', 'Identificador': 'Comuna 1', 'Indicador turismo': 0.001612276785714286, 'Violencia de género': 0.2517208271787297, 'IDH': 0.58625, 'Tasa de desempleo': 0.15861111111111112}, {'Código': 2, 'Nombre': 'Santa Cruz', 'Identificador': 'Comuna 2', 'Indicador turismo': 0.001612276785714286, 'Violencia de género': 0.2054579025110783, 'IDH': 0.6075, 'Tasa de desempleo': 0.13978888888888888}, {'Código': 3, 'Nombre': 'Manrique', 'Identificador': 'Comuna 3', 'Indicador turismo': 0.001612276785714286, 'Violencia de género': 0.2393500738552437, 'IDH': 0.62125, 'Tasa de desempleo': 0.14884444444444445}, {'Código': 4, 'Nombre': 'Aranjuez', 'Identificador': 'Comuna 4', 'Indicador turismo': 0.004836830357142857, 'Violencia de género': 0.20236336779911374, 'IDH': 0.6725, 'Tasa de desempleo': 0.1304888888888889}, {'Código': 5, 'Nombre': 'Castilla', 'Identificador': 'Comuna 5', 'Indicador turismo': 0.00806138392857143, 'Violencia de género': 0.14293205317577548, 'IDH': 0.

In [4]:
# Universe of discourse
x_gender_violence = np.arange(0, 1.01, 0.01)
x_idh = np.arange(0, 1.01, 0.01)
x_unemployment = np.arange(0, 1.01, 0.01)
x_tourism_indicator = np.arange(0, 1.01, 0.01)
x_sustainable_tourism = np.arange(0, 1.01, 0.01)

# Membership functions for each variable

# 1. Gender Violence - Gaussian functions for smooth transitions between levels
gender_violence_low = fuzz.trimf(x_gender_violence, [0, 0, 0.1])
gender_violence_mid = fuzz.trimf(x_gender_violence, [0.05, 0.2, 0.35])
gender_violence_high = fuzz.trimf(x_gender_violence, [0.3, 1, 1])

# 2. Human Development Index (IDH) - Trapezoidal functions for broader ranges at low, medium, and high levels
idh_low = fuzz.trapmf(x_idh, [0, 0, 0.5, 0.6])
idh_mid = fuzz.trimf(x_idh, [0.5, 0.7, 0.9])
idh_high = fuzz.trapmf(x_idh, [0.8, 0.9, 1, 1])

# 3. Unemployment - Mixed trapezoidal and triangular functions to capture low, medium, and high unemployment
unemployment_low = fuzz.trimf(x_unemployment, [0, 0, 0.3])
unemployment_mid = fuzz.trimf(x_unemployment, [0.2, 0.35, 0.5])
unemployment_high = fuzz.trapmf(x_unemployment, [0.4, 0.5, 1, 1])

# 4. Membership functions for Tourism Indicator

# Low tourism indicator uses a trapezoidal function for lower values
tourism_indicator_low = fuzz.trapmf(x_tourism_indicator, [0, 0, 0.3, 0.5])
# Mid tourism indicator is represented by a Gaussian function centered at 0.5
tourism_indicator_mid = fuzz.gaussmf(x_tourism_indicator, 0.5, 0.15)
# High tourism indicator is a trapezoidal function for upper values
tourism_indicator_high = fuzz.trapmf(x_tourism_indicator, [0.5, 0.7, 1, 1])

# 5. Membership functions for Sustainable Tourism (output variable)

# Low sustainable tourism uses a Gaussian function centered at 0
sustainable_tourism_low = lambda y: fuzz.gaussmf(y, 0, 0.2)
# Mid sustainable tourism uses a triangular function for the middle range
sustainable_tourism_mid = lambda y: fuzz.gaussmf(y, 0.5, 0.4)
# High sustainable tourism is represented by a Gaussian function centered at 1
sustainable_tourism_high = lambda y: fuzz.gaussmf(y, 1, 0.2)

In [5]:
def rules_base(input_gender_violence, input_idh, input_unemployment, input_tourism_indicator):

    # Degree of membership for "Gender Violence" input
    gender_violence_fit_low = fuzz.interp_membership(x_gender_violence, gender_violence_low, input_gender_violence)
    gender_violence_fit_mid = fuzz.interp_membership(x_gender_violence, gender_violence_mid, input_gender_violence)
    gender_violence_fit_high = fuzz.interp_membership(x_gender_violence, gender_violence_high, input_gender_violence)

    # Degree of membership for "IDH" input
    idh_fit_low = fuzz.interp_membership(x_idh, idh_low, input_idh)
    idh_fit_mid = fuzz.interp_membership(x_idh, idh_mid, input_idh)
    idh_fit_high = fuzz.interp_membership(x_idh, idh_high, input_idh)

    # Degree of membership for "Unemployment" input
    unemployment_fit_low = fuzz.interp_membership(x_unemployment, unemployment_low, input_unemployment)
    unemployment_fit_mid = fuzz.interp_membership(x_unemployment, unemployment_mid, input_unemployment)
    unemployment_fit_high = fuzz.interp_membership(x_unemployment, unemployment_high, input_unemployment)

    # Degree of membership for "Tourism Indicator" input
    tourism_indicator_fit_low = fuzz.interp_membership(x_tourism_indicator, tourism_indicator_low, input_tourism_indicator)
    tourism_indicator_fit_mid = fuzz.interp_membership(x_tourism_indicator, tourism_indicator_mid, input_tourism_indicator)
    tourism_indicator_fit_high = fuzz.interp_membership(x_tourism_indicator, tourism_indicator_high, input_tourism_indicator)

    # Rule set with combinations of low, mid, and high for each variable

    # Gender Violence (Low, Mid, High) * IDH (Low, Mid, High) * Unemployment (Low, Mid, High) * Tourism Indicator (Low, Mid, High)
    rule1 = gender_violence_fit_low * idh_fit_low * unemployment_fit_low * tourism_indicator_fit_low
    rule2 = gender_violence_fit_low * idh_fit_low * unemployment_fit_low * tourism_indicator_fit_mid
    rule3 = gender_violence_fit_low * idh_fit_low * unemployment_fit_low * tourism_indicator_fit_high
    rule4 = gender_violence_fit_low * idh_fit_low * unemployment_fit_mid * tourism_indicator_fit_low
    rule5 = gender_violence_fit_low * idh_fit_low * unemployment_fit_mid * tourism_indicator_fit_mid
    rule6 = gender_violence_fit_low * idh_fit_low * unemployment_fit_mid * tourism_indicator_fit_high
    rule7 = gender_violence_fit_low * idh_fit_low * unemployment_fit_high * tourism_indicator_fit_low
    rule8 = gender_violence_fit_low * idh_fit_low * unemployment_fit_high * tourism_indicator_fit_mid
    rule9 = gender_violence_fit_low * idh_fit_low * unemployment_fit_high * tourism_indicator_fit_high
    rule10 = gender_violence_fit_low * idh_fit_mid * unemployment_fit_low * tourism_indicator_fit_low
    rule11 = gender_violence_fit_low * idh_fit_mid * unemployment_fit_low * tourism_indicator_fit_mid
    rule12 = gender_violence_fit_low * idh_fit_mid * unemployment_fit_low * tourism_indicator_fit_high
    rule13 = gender_violence_fit_low * idh_fit_mid * unemployment_fit_mid * tourism_indicator_fit_low
    rule14 = gender_violence_fit_low * idh_fit_mid * unemployment_fit_mid * tourism_indicator_fit_mid
    rule15 = gender_violence_fit_low * idh_fit_mid * unemployment_fit_mid * tourism_indicator_fit_high
    rule16 = gender_violence_fit_low * idh_fit_mid * unemployment_fit_high * tourism_indicator_fit_low
    rule17 = gender_violence_fit_low * idh_fit_mid * unemployment_fit_high * tourism_indicator_fit_mid
    rule18 = gender_violence_fit_low * idh_fit_mid * unemployment_fit_high * tourism_indicator_fit_high
    rule19 = gender_violence_fit_low * idh_fit_high * unemployment_fit_low * tourism_indicator_fit_low
    rule20 = gender_violence_fit_low * idh_fit_high * unemployment_fit_low * tourism_indicator_fit_mid
    rule21 = gender_violence_fit_low * idh_fit_high * unemployment_fit_low * tourism_indicator_fit_high
    rule22 = gender_violence_fit_low * idh_fit_high * unemployment_fit_mid * tourism_indicator_fit_low
    rule23 = gender_violence_fit_low * idh_fit_high * unemployment_fit_mid * tourism_indicator_fit_mid
    rule24 = gender_violence_fit_low * idh_fit_high * unemployment_fit_mid * tourism_indicator_fit_high
    rule25 = gender_violence_fit_low * idh_fit_high * unemployment_fit_high * tourism_indicator_fit_low
    rule26 = gender_violence_fit_low * idh_fit_high * unemployment_fit_high * tourism_indicator_fit_mid
    rule27 = gender_violence_fit_low * idh_fit_high * unemployment_fit_high * tourism_indicator_fit_high
    rule28 = gender_violence_fit_mid * idh_fit_low * unemployment_fit_low * tourism_indicator_fit_low
    rule29 = gender_violence_fit_mid * idh_fit_low * unemployment_fit_low * tourism_indicator_fit_mid
    rule30 = gender_violence_fit_mid * idh_fit_low * unemployment_fit_low * tourism_indicator_fit_high
    rule31 = gender_violence_fit_mid * idh_fit_low * unemployment_fit_mid * tourism_indicator_fit_low
    rule32 = gender_violence_fit_mid * idh_fit_low * unemployment_fit_mid * tourism_indicator_fit_mid
    rule33 = gender_violence_fit_mid * idh_fit_low * unemployment_fit_mid * tourism_indicator_fit_high
    rule34 = gender_violence_fit_mid * idh_fit_low * unemployment_fit_high * tourism_indicator_fit_low
    rule35 = gender_violence_fit_mid * idh_fit_low * unemployment_fit_high * tourism_indicator_fit_mid
    rule36 = gender_violence_fit_mid * idh_fit_low * unemployment_fit_high * tourism_indicator_fit_high
    rule37 = gender_violence_fit_mid * idh_fit_mid * unemployment_fit_low * tourism_indicator_fit_low
    rule38 = gender_violence_fit_mid * idh_fit_mid * unemployment_fit_low * tourism_indicator_fit_mid
    rule39 = gender_violence_fit_mid * idh_fit_mid * unemployment_fit_low * tourism_indicator_fit_high
    rule40 = gender_violence_fit_mid * idh_fit_mid * unemployment_fit_mid * tourism_indicator_fit_low
    rule41 = gender_violence_fit_mid * idh_fit_mid * unemployment_fit_mid * tourism_indicator_fit_mid
    rule42 = gender_violence_fit_mid * idh_fit_mid * unemployment_fit_mid * tourism_indicator_fit_high
    rule43 = gender_violence_fit_mid * idh_fit_mid * unemployment_fit_high * tourism_indicator_fit_low
    rule44 = gender_violence_fit_mid * idh_fit_mid * unemployment_fit_high * tourism_indicator_fit_mid
    rule45 = gender_violence_fit_mid * idh_fit_mid * unemployment_fit_high * tourism_indicator_fit_high
    rule46 = gender_violence_fit_mid * idh_fit_high * unemployment_fit_low * tourism_indicator_fit_low
    rule47 = gender_violence_fit_mid * idh_fit_high * unemployment_fit_low * tourism_indicator_fit_mid
    rule48 = gender_violence_fit_mid * idh_fit_high * unemployment_fit_low * tourism_indicator_fit_high
    rule49 = gender_violence_fit_mid * idh_fit_high * unemployment_fit_mid * tourism_indicator_fit_low
    rule50 = gender_violence_fit_mid * idh_fit_high * unemployment_fit_mid * tourism_indicator_fit_mid
    rule51 = gender_violence_fit_mid * idh_fit_high * unemployment_fit_mid * tourism_indicator_fit_high
    rule52 = gender_violence_fit_mid * idh_fit_high * unemployment_fit_high * tourism_indicator_fit_low
    rule53 = gender_violence_fit_mid * idh_fit_high * unemployment_fit_high * tourism_indicator_fit_mid
    rule54 = gender_violence_fit_mid * idh_fit_high * unemployment_fit_high * tourism_indicator_fit_high
    rule55 = gender_violence_fit_high * idh_fit_low * unemployment_fit_low * tourism_indicator_fit_low
    rule56 = gender_violence_fit_high * idh_fit_low * unemployment_fit_low * tourism_indicator_fit_mid
    rule57 = gender_violence_fit_high * idh_fit_low * unemployment_fit_low * tourism_indicator_fit_high
    rule58 = gender_violence_fit_high * idh_fit_low * unemployment_fit_mid * tourism_indicator_fit_low
    rule59 = gender_violence_fit_high * idh_fit_low * unemployment_fit_mid * tourism_indicator_fit_mid
    rule60 = gender_violence_fit_high * idh_fit_low * unemployment_fit_mid * tourism_indicator_fit_high
    rule61 = gender_violence_fit_high * idh_fit_low * unemployment_fit_high * tourism_indicator_fit_low
    rule62 = gender_violence_fit_high * idh_fit_low * unemployment_fit_high * tourism_indicator_fit_mid
    rule63 = gender_violence_fit_high * idh_fit_low * unemployment_fit_high * tourism_indicator_fit_high
    rule64 = gender_violence_fit_high * idh_fit_mid * unemployment_fit_low * tourism_indicator_fit_low
    rule65 = gender_violence_fit_high * idh_fit_mid * unemployment_fit_low * tourism_indicator_fit_mid
    rule66 = gender_violence_fit_high * idh_fit_mid * unemployment_fit_low * tourism_indicator_fit_high
    rule67 = gender_violence_fit_high * idh_fit_mid * unemployment_fit_mid * tourism_indicator_fit_low
    rule68 = gender_violence_fit_high * idh_fit_mid * unemployment_fit_mid * tourism_indicator_fit_mid
    rule69 = gender_violence_fit_high * idh_fit_mid * unemployment_fit_mid * tourism_indicator_fit_high
    rule70 = gender_violence_fit_high * idh_fit_mid * unemployment_fit_high * tourism_indicator_fit_low
    rule71 = gender_violence_fit_high * idh_fit_mid * unemployment_fit_high * tourism_indicator_fit_mid
    rule72 = gender_violence_fit_high * idh_fit_mid * unemployment_fit_high * tourism_indicator_fit_high
    rule73 = gender_violence_fit_high * idh_fit_high * unemployment_fit_low * tourism_indicator_fit_low
    rule74 = gender_violence_fit_high * idh_fit_high * unemployment_fit_low * tourism_indicator_fit_mid
    rule75 = gender_violence_fit_high * idh_fit_high * unemployment_fit_low * tourism_indicator_fit_high
    rule76 = gender_violence_fit_high * idh_fit_high * unemployment_fit_mid * tourism_indicator_fit_low
    rule77 = gender_violence_fit_high * idh_fit_high * unemployment_fit_mid * tourism_indicator_fit_mid
    rule78 = gender_violence_fit_high * idh_fit_high * unemployment_fit_mid * tourism_indicator_fit_high
    rule79 = gender_violence_fit_high * idh_fit_high * unemployment_fit_high * tourism_indicator_fit_low
    rule80 = gender_violence_fit_high * idh_fit_high * unemployment_fit_high * tourism_indicator_fit_mid
    rule81 = gender_violence_fit_high * idh_fit_high * unemployment_fit_high * tourism_indicator_fit_high

    rules = [rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10,
    rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18, rule19, rule20,
    rule21, rule22, rule23, rule24, rule25, rule26, rule27, rule28, rule29, rule30,
    rule31, rule32, rule33, rule34, rule35, rule36, rule37, rule38, rule39, rule40,
    rule41, rule42, rule43, rule44, rule45, rule46, rule47, rule48, rule49, rule50,
    rule51, rule52, rule53, rule54, rule55, rule56, rule57, rule58, rule59, rule60,
    rule61, rule62, rule63, rule64, rule65, rule66, rule67, rule68, rule69, rule70,
    rule71, rule72, rule73, rule74, rule75, rule76, rule77, rule78, rule79, rule80,
    rule81]

    return rules 


In [6]:
def fuzzification(rules):
    out_sustainable_tourism_low = np.fmax(np.fmax(np.fmax(
        np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
            np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                    np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                        np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                            np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                                np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                                    np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                                        np.fmax(np.fmax(np.fmax(                                                   
                                            rules[2], rules[3]), rules[4]), rules[5]), rules[6]), rules[7]),
                                            rules[8]), rules[9]), rules[12]), rules[15]), rules[16]),
                                            rules[17]), rules[18]), rules[19]), rules[27]), rules[28]),
                                            rules[29]), rules[30]), rules[31]), rules[32]), rules[33]),
                                            rules[34]), rules[35]), rules[36]), rules[43]), rules[44]),
                                            rules[51]), rules[54]), rules[55]), rules[56]), rules[57]),
                                            rules[58]), rules[59]), rules[60]), rules[61]), rules[62]),
                                            rules[63]), rules[64]), rules[65]), rules[66]), rules[67]),
                                            rules[68]), rules[69]), rules[70]), rules[71]), rules[72]),
                                            rules[73]), rules[74]), rules[75]), rules[76]), rules[77]),
                                            rules[78]), rules[79]), rules[80])
    
    out_sustainable_tourism_mid = np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                                    np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                                        np.fmax(np.fmax(np.fmax(                                                   
                                            rules[0], rules[1]), rules[9]), rules[10]), rules[14]), rules[21]),
                                            rules[24]), rules[37]), rules[38]), rules[39]), rules[40]),
                                            rules[41]), rules[42]), rules[45]), rules[46]), rules[47]),
                                            rules[48]), rules[49]), rules[50]), rules[51])
    
    out_sustainable_tourism_high = np.fmax(np.fmax(np.fmax(np.fmax(
                                        np.fmax(np.fmax(np.fmax(np.fmax(                                                   
                                            rules[11], rules[13]), rules[19]), rules[20]), rules[22]), rules[23]),
                                            rules[25]), rules[26]), rules[53])

    fuzzy_outputs = [out_sustainable_tourism_low, out_sustainable_tourism_mid, out_sustainable_tourism_high]
    
    return fuzzy_outputs

In [7]:
def defuzzification(fuzzy_outputs):
    # Defuzzification with centroid, bisector and mean of maximum methods
    fuzzy_out_sustainable_tourism = np.fmax(np.fmax(np.fmin(fuzzy_outputs[0], sustainable_tourism_low(x_sustainable_tourism)), 
                                            np.fmin(fuzzy_outputs[1], sustainable_tourism_mid(x_sustainable_tourism))),
                                            np.fmin(fuzzy_outputs[2], sustainable_tourism_high(x_sustainable_tourism)))
    
    defuzzified_sustainable_tourism  = fuzz.defuzz(x_sustainable_tourism, fuzzy_out_sustainable_tourism, 'centroid')

    return defuzzified_sustainable_tourism

In [8]:
def eval_fuzzy_system(input_gender_violence, input_unemployment, input_idh, input_tourism):
    rules = rules_base(input_gender_violence, input_idh, input_unemployment, input_tourism)
    fuzzy_outputs = fuzzification(rules)
    outputs = defuzzification(fuzzy_outputs)
    return outputs

In [9]:
results = {}

# Calculate the sustainable tourism indicator
for codigo_comuna, datos_comuna in commune_data.items():
    input_gender_violence = datos_comuna['Violencia de género']
    input_unemployment = datos_comuna['Tasa de desempleo']
    input_idh = datos_comuna['IDH']
    input_tourism = datos_comuna['Indicador turismo']
    
    # Execute fuzzy logic model
    sustainable_tourism = eval_fuzzy_system(input_gender_violence, input_unemployment, input_idh, input_tourism)
    
    # Save results
    results[codigo_comuna] = sustainable_tourism

    results_list = [{'Código': codigo_comuna, 'Turismo sostenible': resultado} 
                    for codigo_comuna, resultado in results.items()]

    df_resultados = pd.DataFrame(results_list)
    df_resultados.to_csv('Socioeconomical_tourism_model.csv', index=False)